# CUWALID MODEL TRAINING

## Hydrological model DRYP

The following course cover the following content:

* Installation
* Preparing model input parameters and dataset
* ***Preparing DRYP simulations***
* Post processing model outputs


### 3. Preparing DRYP model simulations

1. Preparing model parameter and setting files
* Running DRYP
* Preparing multiple simulation
* Runing pipeline DRYP


In [ ]:
print("Local_directory\n|__training\n      |__basin\n      |   |__datasets\n      |   |    |__csv\n      |   |    |__shp\n      |   |__model\n      |   |    |__inputs\n      |   |__outputs\n      |__regional")

Specifiy the location of training files, once the training files are downloaded change the paths below to access them through this traning material.

In [ ]:

training_general_path = "D:/HAD/training/"
regional_path = "D:/HAD/training/regional/"
basin_path = "D:/HAD/training/basin/"


#### 3.1. Preparing model parameter and setting files

DRYP requires at leas two files for running, this is the input-paramter-file and the setting-paramter file. Additional
files may be required depending on the model settings and dataset properties.

The folowing are the list of DRYP model files:

* input parameter file (essential)
* setting-paramter file (essential)
* GW parameter file (optional)
* riparian input paramter file (optional)
* data projection setting file (optional)

Model input and settins files are simple plain text format files, so they can be easily updated using any text editor.

For conveniency we use here the library pandas to edit these files.

In [ ]:
import pandas as pd

* Input parameter file

The *input-parameter-file* is used for specifying path names of all names model files, including input parameters, forcings, and setting files, as well as model outputs. You can also specified the model name as well as any additional model file.

In [ ]:
finput = training_general_path + "/regional/model/HAD_IMERG_Tana_input_sim.dmp"

In [ ]:
dryp_input = pd.read_csv(finput)

In [ ]:
dryp_input.info()

In [ ]:
dryp_input.head(5)

Surface parameters

In [ ]:
#dryp_input.iloc[3:19]

Soil parameters

In [ ]:
#dryp_input.iloc[26:49]

Groundwater parameters

In [ ]:
#dryp_input.iloc[50:64]

Forcing datasets, meteorological data

In [ ]:
dryp_input.iloc[64:69]

* Main model setting file

In [ ]:
#dryp_input.tail(12)

In [ ]:
fsettings = training_general_path + "/regional/model/HAD_IMERG_Tana_par_setting.dwapm"

In [ ]:
dryp_settings = pd.read_csv(fsettings)
dryp_settings.head(10)

* Model setting file: Reference and projection system

In [ ]:
fprojection = training_general_path + "/regional/model/HAD_projection.dwapm"

In [ ]:
dryp_projection = pd.read_csv(fprojection)

* Riparian input setting files

In [ ]:
friparian = basin_path + "/model/HAD_riparian_inputs.txt"


In [ ]:
dryp_riparian = pd.read_csv(friparian)

* Groundwater parameter file

In [ ]:
fGWparameters = training_general_path + "/regional/model/HAD_GW_parameters.dwapm"

In [ ]:
dryp_GWparamters = pd.read_csv(fGWparameters)

* Changing model input paths

Creating a new model requires to specify the new paths of the model parameter files, it is a task that takes time and can be prone to errors. To avoid any mistake when changing paths, we can use the replace text option availble in python.

In [ ]:
from shutil import copyfile
import fileinput
import sys

In [ ]:
def replaceAll(file, searchExp, replaceExp):
    for line in fileinput.input(file, inplace=1):
        if searchExp in line:
            line = line.replace(searchExp,replaceExp)
        sys.stdout.write(line)

In [ ]:
fname_file = regional_path + "/model/HAD_IMERG_input_sim_40.dmp"
fname_file_update = regional_path + "/model/HAD_IMERG_input_sim_cuwalid.dmp"

Create a copy avoid the lost of information

In [ ]:
copyfile(fname_file, fname_file_update)

In [ ]:
#pd.read_csv(fname_file_update)

Specify the which text you want to change and the new text

In [ ]:
searchExpresion = "/home/c1755103/HAD/input/"
replaceExpresion = "/home/cuwalid/HAD/input/"
replaceAll(fname_file_update, searchExpresion, replaceExpresion)

In [ ]:
searchExpresion = "/home/c1755103/HAD/HAD_"
replaceExpresion = "/home/cuwalid/HAD/HAD_"
replaceAll(fname_file_update, searchExpresion, replaceExpresion)

In [ ]:
#pd.read_csv(fname_file_update)

#### 3.2. Runing DRYP

**WARNING**: If you are using Windows you have to use the docker for running DRYP

There are two main ways of runing DRYP, you can even add more option that are more convenient for yor analysis:

a) Running DRYP from the command line

In [ ]:
# python run_model_input.py <finput>

b) Running dryp within the python environment

In [ ]:
import sys
sys.path.append('C:/Users/Edisson/Documents/GitHub/DRYPv2.0.1')

from dryp.main_DRYP import run_DRYP
#run_DRYP(inputfile)

#### 3.3. Preparing and running multiple DRYP simulations

This is useful for performing a Monte Carlo analysis or stochastic forecasting.

We can create a script to modify the model files that changes specific fiels of each of the all model files, below ther is
an example of the script to genereate multiple simulation files for a calibration pourposes.

In [ ]:
import os
import numpy as np
import pandas as pd

def write_sim_file(fname_input, newfname_input, parameter):
	""" modify the parematers of the model input file and
		model setting file.
		WARNING: it will reeplace the original file, so
		make a copy of the original files
	parameters:
		filename_input:	model inputfile, including path
		parameter:		1D array of model paramters
	"""
	
	# Check if input file exist
	if not os.path.exists(fname_input):
		raise ValueError("File not availble")
	
	# simulation number
	sim = str(int(parameter[0]))
	
	# Read model input file
	f = pd.read_csv(fname_input)
	
	# Change model name by adding the simulation number
	f.drylandmodel[1] = f.drylandmodel[1] + sim

	# change parameters of the of the setting parameter file
	fname_settings = f.drylandmodel[87]
	# create a copy of the file with the new name
	fname_root = fname_settings.split('.')[0]
	fname_ext = fname_settings.split('.')[1]
	# new input file name
	newfname_settings = fname_root+'_'+sim+'.'+fname_ext
	#copyfile(fname_settings, newfname_settings)
	# replace new setting file
	f.drylandmodel[87] = newfname_settings
	# Open setting parameter file
	fsimpar = pd.read_csv(fname_settings)
	# ADD CODE HERE TO NODIFY FORCING DATSET NAMES
	#f.drylandmodel[66] = newfname_settings
	#f.drylandmodel[68] = newfname_settings
	# Change setting parameter file with new values
	fsimpar['DWAPM_SET'][46] = ('%.5f' % parameter[1]) # kdt
	fsimpar['DWAPM_SET'][48] = ('%.5f' % parameter[2]) # kDroot
	fsimpar['DWAPM_SET'][50] = ('%.2f' % parameter[3]) # kAWC
	fsimpar['DWAPM_SET'][52] = ('%.5f' % parameter[4]) # kKsat
	fsimpar['DWAPM_SET'][54] = ('%.5f' % parameter[5]) # kSigma
	fsimpar['DWAPM_SET'][56] = ('%.5f' % parameter[6]) # kKch
	fsimpar['DWAPM_SET'][58] = ('%.5f' % parameter[7]) # T
	fsimpar['DWAPM_SET'][60] = ('%.5f' % parameter[8]) # kW
	fsimpar['DWAPM_SET'][62] = ('%.5f' % parameter[9]) # kKaq
	fsimpar['DWAPM_SET'][64] = ('%.5f' % parameter[10])# kSy
	
	# Reeplace model input and parameters file
	os.remove(newfname_input) if os.path.exists(newfname_input) else None
	os.remove(newfname_settings) if os.path.exists(newfname_settings) else None
	
	# Write model parameter and input file
	f.to_csv(newfname_input, index=False)
	fsimpar.to_csv(newfname_settings, index=False)

def gen_array_input_files(fname_input, fname_parameter_sets):
	# read parameter sets
	parameter = pd.read_csv(fname_parameter_sets)
	
	#Create a copy of inputfile
	fname_root = fname_input.split('.')[0]
	fname_ext = fname_input.split('.')[1]
	for npar in range(0, 100):
		# new input file name
		newfname_input = fname_root+'_'+str(npar)+'.'+fname_ext
		# replace all new values in dataset
		write_sim_file(fname_input, newfname_input, parameter.loc[npar])


In [ ]:
# ========================================================
# LOOP FOR CREATING MULTIPLE IMPORT FILES FOR RUNNING IN HPC
fname = [
basin_path +'model/HAD_IMERG_Tana_input_sim.dmp',
]
fname_parameter_sets = training_general_path + "/basin/dataset/csv/test_parameter_set.csv"
for ifname in fname:
	gen_array_input_files(ifname, fname_parameter_sets)

For running the script above you need to create a parameter set file, an example is shown below:

In [ ]:
fname_parameter_sets = training_general_path + "/regional/datasets/csv/test_parameter_set.csv"

In [ ]:
pd.read_csv(fname_parameter_sets).head(5)

**TASK**: modify the script to create model files for running multiples simulation with different forcing
datasets (e.g. stochastic forecasting).

**HINT**: lines 66 and 68 needs to be modified in the input model parameters file, add lines in script to modify this lines

#### 3.4. Runing pipeline DRYP simulations

For large simulations output files can become very large which may result in consuming cosiderable amount of
memory that can even stop the model. This problem is very challenging to address in python, requiring significan changes
in the code or even considering parallelisation. Here, we simply split a continuos simulation (in time, not in space)
to store model output files in specified simulation periods (e.g. years).

For this, we can create a series of model files that read and save inital conditions for the previous and subsequente
simulations. An example of a python script to generate pipeline simulaiton is presented below:

In [ ]:
import os
import numpy as np
import pandas as pd

def write_sim_file_pipeline(fname_input, newfname_input,
	sim_ini='1999', sim_end='2000',
	date_ini='2000 1 1', date_end='2001 1 1'):
	""" modify the parematers of the model input file and
		model setting file.
		WARNING: it will reeplace the original file, so
		make a copy of the original files
	parameters:
		filename_input:	model inputfile, including path
		parameter:		1D array of model paramters
	"""
	
	# Check if input file exist
	if not os.path.exists(fname_input):
		raise ValueError("File not availble")
	
	#date_ini = 
	#date_end = str
	
	# simulation number
	#sim = str(int(parameter[0]))
	
	# Read model input file
	f = pd.read_csv(fname_input)
	
	# copy input file
	fnew = f.copy()
	# INPUT FILES ====================================
	# Change model name by adding the simulation number
	fnew.drylandmodel[1] = f.drylandmodel[1][:-4] + sim_end
	
	# name and directory of previous time steps
	mname = f.drylandmodel[1][:-4] + sim_ini
	DirOutput = f.drylandmodel[81]# + '_' + sim_ini
	
	# new name
	fnew.drylandmodel[6] = f.drylandmodel[1] + '_' + sim_ini
	
	# change initial conditions Qo
	fnew.drylandmodel[6] = DirOutput+'/'+mname+'_avg_Q_ini.asc'
	# change initial conditions soil moisture
	fnew.drylandmodel[46] = DirOutput+'/'+mname+'_avg_tht_ini.asc'
	# change initial condition water table
	fnew.drylandmodel[57] = DirOutput+'/'+mname+'_avg_wte_ini.asc'
	
	# RIPARIAN FILES ==================================
	# change files of the riparian file	
	fname_riparian = f.drylandmodel[93]
	# create a copy of the file with the new name
	fname_rootrp = fname_riparian.split('.')[0]
	fname_extrp = fname_riparian.split('.')[1]
	# new input file name
	newfname_riparian = fname_rootrp + '_' + sim_end + '.' + fname_extrp
	# change initial condition riparian area
	# change initial conditions riparian water content
	fsimrip = pd.read_csv(fname_riparian)
	fsimrip.RIPARIAN[19] = DirOutput+'/'+mname+'_avg_tht_rp_ini.asc'
	
	# write name of riparian file in the input file
	fnew.drylandmodel[93] = newfname_riparian
	
	# SETTING FILES ===================================
	# change parameters of the of the setting parameter file	
	fname_settings = f.drylandmodel[87]
	# create a copy of the file with the new name
	fname_root = fname_settings.split('.')[0]
	fname_ext = fname_settings.split('.')[1]
	# new input file name
	newfname_settings = fname_root + '_' + sim_end + '.' + fname_ext
	#copyfile(fname_settings, newfname_settings)
	# replace new setting file
	fnew.drylandmodel[87] = newfname_settings
	
	
	# Open setting parameter file
	fsimpar = pd.read_csv(fname_settings)	
	
	# Change starting point of the simulation
	fsimpar.DWAPM_SET[2] = date_ini
		
	# change final date of the simulaition
	fsimpar.DWAPM_SET[4] = date_end

	# Reeplace model input and parameters file
	os.remove(newfname_input) if os.path.exists(newfname_input) else None
	os.remove(newfname_riparian) if os.path.exists(newfname_riparian) else None
	os.remove(newfname_settings) if os.path.exists(newfname_settings) else None
	
	# Write model parameter and input file
	fnew.to_csv(newfname_input, index=False)
	fsimrip.to_csv(newfname_riparian, index=False)
	fsimpar.to_csv(newfname_settings, index=False)

def change_sim_file_name(fname_input, newfname_input, sim=2000):
	# Check if input file exist
	if not os.path.exists(fname_input):
		raise ValueError("File not availble")
	
	f = pd.read_csv(fname_input)
	f.drylandmodel[1] = f.drylandmodel[1] + "_" + str(sim)
	# Reeplace model input and parameters file
	os.remove(newfname_input) if os.path.exists(newfname_input) else None
	# Write model parameter and input file
	f.to_csv(newfname_input, index=False)

def gen_inital_end_simulation_dates(year, month, day, dtyear=1, dtmonth=0, dtday=0):
	"""This function gets the initial and end date of a specified period
	"""
	date_ini = str(year+dtyear) + ' ' + str(month+dtmonth) + ' ' + str(day+dtday)
	date_end = str(year+dtyear+1) + ' ' + str(month+dtmonth) + ' ' + str(day+dtday+1)
	
	name = str(year)
	name_end = str(year+dtyear)
	
	return date_ini, date_end, name, name_end

In [ ]:
# ========================================================
# LOOP FOR CREATING MULTIPLE IMPORT FILES FOR RUNNING IN HPC
fname = [
training_general_path + "model/HAD_IMERG_Tana_input_sim.dmp",
]

# WARNINGS
# The initial file should specify the initial conditions at the begining of
# the simulation of the entire period, therefore, the name of inital model
# must be according to the name required to the next simulation
# subsequent period must conside
for ifname_input in fname:
	newfname_input = ifname_input.split('.')[0]+'_2000.'+ifname_input.split('.')[1]
	change_sim_file_name(ifname_input, newfname_input)
	for iyear in range(2000, 2023):
		date_ini, date_end, name, name_end = gen_inital_end_simulation_dates(iyear, 1, 1)
		newfname_input = ifname_input.split('.')[0]+'_'+name_end+'.'+ifname_input.split('.')[1]
		write_sim_file_pipeline(ifname_input, newfname_input, sim_ini=name, sim_end=name_end,
			date_ini=date_ini, date_end=date_end
			)

In [ ]:
import glob
files = glob.glob(regional_path + "inputs/*.asc")

In [ ]:
new_path = "D:/"
fnames = []
for ifile in files:
    fnames.append(ifile.split("\\")[-1])
    new_name = new_path + ifile.split("\\")[-1]
    print(new_name)
    #new_name = new_path + fnames[0]
    #process clip raster

In [ ]:
#fnames

In [ ]:
#new_path = "D:/"
#new_name = new_path + fnames[0]